## Extract NDVI (Landsat - 8)

In [33]:
import ee
import folium
import geemap

In [65]:
# Authenticate
ee.Authenticate()

True

In [35]:
# Initialize Earth Engine
ee.Initialize(project='ee-gisengineer17')

In [ ]:
# Define AOI
aoi = ee.Geometry.Rectangle([78.5, 20.5, 79.5, 21.5])

#### 1. Landsat 8 Collection 2, Tier 1, TOA (Top of Atmosphere) reflectance data.
#### 2. Landsat 8 Bands:
B1: Coastal/Aerosol (0.43-0.45 µm)

B2: Blue (0.45-0.51 µm)

B3: Green (0.53-0.59 µm)

B4: Red (0.64-0.67 µm)

B5: Near Infrared (NIR) (0.85-0.88 µm)

B6: Shortwave Infrared 1 (SWIR1) (1.57-1.65 µm)

B7: Shortwave Infrared 2 (SWIR2) (2.11-2.29 µm)

B8: Panchromatic (0.50-0.68 µm, high-resolution 15m)

B9: Cirrus (1.36-1.38 µm)

B10, B11: Thermal Infrared (TIR1, TIR2)

In [80]:
# Load Landsat-8 Image Collection
image = (ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")  # Landsat 8 TOA Reflectance.
         .filterBounds(aoi)  # Filter by AOI.
         .filterDate('2013-04-11', '2024-03-10')   # Landsat-8 available from April 2013.
         .filter(ee.Filter.lt('CLOUD_COVER', 10))  # Filter low cloud cover, Helps improve NDVI accuracy by removing cloudy pixels.
         .median()  # Merge images to reduce gaps, Helps remove clouds and shadows, Reduces noise from atmospheric conditions. 
         .clip(aoi))  # Clip to AOI

NDVI values range from -1 to +1:

NDVI values range from -1 to +1:

+0.6 to +1.0 → Dense green vegetation

+0.2 to +0.5 → Sparse vegetation 

Negative values → Water, snow, or built-up areas

#### The .normalizedDifference(['B5', 'B4']) method in Google Earth Engine (GEE) automatically applies:

In [74]:
# Compute NDVI: (NIR - Red) / (NIR + Red)
ndvi = image.normalizedDifference(['B5', 'B4']).rename("NDVI") 

In [68]:
# Map visualization parameters
ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

In [79]:
# Create a map
m = geemap.Map(center=[21.0, 79.0], zoom=9)
m.addLayer(aoi, {"color": "red"}, "Bounding Box")
m.addLayer(ndvi, ndvi_vis, "NDVI from Landsat-8")
m

Map(center=[21.0, 79.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

#### Available image count

In [64]:
count = (ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
         .filterBounds(aoi)
         .filterDate('2013-04-11', '2024-03-10')
         .size().getInfo())
print(f"Available Landsat-8 images: {count}")

Available Landsat-8 images: 616
